# Assoziaionsanlayse

Untersucht wird nur der Datensatz der Rotweine (~1600 Zeilen).
Der Datensatz der Weißweine ist zu groß (~5000 Zielen).

## Setup

In [ ]:
# Setup
from mlxtend.frequent_patterns import apriori, association_rules
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.plotting

In [ ]:
# Info über Spalten (detusche Übersetzungen, ...)
columns = {
    "fixed acidity": {
        "de": "Fester Säuregehalt [g/L]",
    },
    "volatile acidity": {
        "de": "Flüchtiger Säuregehalt [g/L]",
    },
    "citric acid": {
        "de": "Citronensäure [g/L]",
    },
    "residual sugar": {
        "de": "Restzucker [g/L]",
        "fixed_filter": 10.0,
    },
    "chlorides": {
        "de": "Chloride [g/L]",
        "fixed_filter": 0.3,
    },
    "free sulfur dioxide": {
        "de": "Freie Schwefeloxide [mg/L]",
    },
    "total sulfur dioxide": {
        "de": "Schwefeloxide [mg/L]",
    },
    "density": {
        "de": "Dichte [g/cm3]",
    },
    "pH": {
        "de": "pH-Wert",
    },
    "sulphates": {
        "de": "Sulfate [g/L]",
    },
    "alcohol": {
        "de": "Alkohol [vol.%]",
    },
    "quality": {
        "de": "Qualität",
    },
}

In [ ]:
# Einelsen der Daten
df_red = pd.read_csv('winequality-red.csv', sep=';', header=0)

## Bereinigung der Daten

### Null-Werte

In [ ]:
# Null-Werte prüfen
print(df_red.isna().sum().sum())

### Ausreißer

In [ ]:
def hist(df):
    fig = plt.figure(
        num="Histogramm",
        figsize=(12,9),
    )
    fig.subplots_adjust(hspace=0.4, wspace=0.5)

    for i, column in enumerate(columns.keys()):
        f = fig.add_subplot(4, 3, i+1)
        f.set_title(columns[column]["de"])
        f.hist(df[column], bins=20)
        f.axvline(df[column].mean(), color='k', linestyle='dashed', linewidth=1)
        # TODO: print outliers red
        # (https://stackoverflow.com/questions/49290266/python-matplotlib-histogram-specify-different-colours-for-different-bars)

    plt.show()

def box_plot(df):
    fig = plt.figure(
        num="Box Plot",
        figsize=(12,9),
    )
    fig.subplots_adjust(hspace=0.4, wspace=0.5)

    for i, column in enumerate(columns.keys()):
        f = fig.add_subplot(4, 3, i+1)
        f.set_title(columns[column]["de"])
        f.boxplot(df[column], vert=False)

    plt.show()

In [ ]:
hist(df_red)
box_plot(df_red)

In [ ]:
def print_outliers(df):
    # Z-Score ist die Abweichung vom Mittelwert in Standardabweichungen
    # betragsmäßig hoher Z-Score deutet auf Ausreißer hin.
    z = np.abs(stats.zscore(df))

    for i, column in enumerate(columns):
        print("\n")
        print(column)

        filter = (z[column] > 2.5)
        if "fixed_filter" in columns[column].keys():
            filter = (df[column] > columns[column]["fixed_filter"])
        print(df[filter].sort_values(by=column))


def remove_outliers(df, max_z_score):
    z = np.abs(stats.zscore(df))
    filter = (z < max_z_score).all(axis=1)
    return df[filter]

In [ ]:
print_outliers(df_red)

# Datensatz bereinigen
#df = remove_outliers(df_red, 2.5)
#hist(df)
#df.to_csv('./winequality-red-filtered.csv', sep=';', header=True, index=False)

### Ergebnisse

Es gibt viele Ausreißer.
Fehlmessungen werden aufgrund physikalischer Zusammenhänge einiger Spalten ausgeschlossen.
Die Ausreißer sind vermutlich der Tatsache geschuldet, dass Wein ein Naturprodukt ist.

In [ ]:
# Extrem hohe Citronensäure, sehr hoher Alkohlgehalt, sehr niedriger PH-Wert
print(df_red.loc[652])

# Extrem hohe Citronensäure, sehr niedriger PH-Wert
# Extrem hohe Chlorid-Werte üblich bei Meernahen Weinanbaugebieten wie Portugal
# https://www.institut-heidger.de/anionen/.
print(df_red.loc[151])

## Zusammenhänge
- Zusammenhang: Feste Säure - Citronensäure - PH-Wert - Dichte
- Zusammenhang: Schwefeloxide - freie Schwefeloxide
- Zusammenhang: Flüchtige Säuren - Akohol - Qualität

In [ ]:
def scatter(df, columns):
    pd.plotting.scatter_matrix(df[columns],
        figsize=(15, 15),
        marker="o",
        c=df['quality'].values,
        s=30,
        alpha=0.8,
    )
    plt.show()

In [ ]:

scatter(df_red, ["fixed acidity","citric acid","density","pH"])
scatter(df_red, ["free sulfur dioxide","total sulfur dioxide"])
scatter(df_red, ["volatile acidity","alcohol","quality"])

In [67]:
# Einteilung in je 3 Klassen (unter 25%, 25-75%, ueber 75% Quantil)
# ==> Binäre Matrix für mlextends Funktionen
df = pd.DataFrame()
for c in columns.keys():
    q_25 = df_red[c].quantile(q=0.25)
    q_75 = df_red[c].quantile(q=0.75)
    print(f'{c}:  {q_25}  {q_75}')
    #df[f'{c}_low'] = df_red[c].le(q_25).astype(int)
    #df[f'{c}_mid'] = df_red[c].between(q_25, q_75, inclusive='right').astype(int)
    df[f'{c}_low'] = df_red[c].le(q_75).astype(int)
    df[f'{c}_high'] = df_red[c].gt(q_75).astype(int)

fixed acidity:  7.1  9.2
volatile acidity:  0.39  0.64
citric acid:  0.09  0.42
residual sugar:  1.9  2.6
chlorides:  0.07  0.09
free sulfur dioxide:  7.0  21.0
total sulfur dioxide:  22.0  62.0
density:  0.9956  0.997835
pH:  3.21  3.4
sulphates:  0.55  0.73
alcohol:  9.5  11.1
quality:  5.0  6.0


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

#TODO
# support min 5% (sonst nicht aussagekräftig)
# alle kombinationen aus 2/3 klassen und max_len=2/3/4
# hohe konfidenz
# hoher lift
# lift bei 1
# kleiner lift

In [62]:
# hohe konfidenz
""" Ergebnis:
- zusammenhang niedriger freier schwefelanteil und niedriger gesamtschwefelanteil
- wenig alkohol -> niedrigere qualitaet (wenig alkhol, weniger suesse trauben, billiger)
- wenig sulfate -> niedrigere qualitaet
- viel schwefel -> niedrigere qualitaet (schwefel = haltbarkeitsmittel, mehr noetig bei faulen trauben)
- hohe fluechtige saeureanteile -> niedrigere qualitaet
- zusammenhang hohe citronensaeuere und viel feste saeure
- zusammenhang ph-wert saeure
- zusammenhang freier und gesamt schwefel
- zusammenhang hoher alkohol -> geringe dichte
==> zusammenhaenge bestaetigt
==> neu: kriterien fuer schlechten wein gefunden (lift allerdings oft nicht hoch)

2 Klassen:
- hohe dichte -> alkohol low und geringere qualitaet
- wenig citronensaeure -> qualiaet low (geringer lift)
- ph high -> qualitaet low (geringer lift)
"""
# Auch mit max_len=3 und head(30) getestet
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules.sort_values(by="confidence", ascending=False, inplace=True)
print(rules.head(20))

                     antecedents                 consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction
68          (fixed acidity_high)                    (pH_low)            0.247655            0.757974  0.240150    0.969697  1.279328  0.052434    7.986867
31                     (pH_high)         (fixed acidity_low)            0.242026            0.752345  0.234522    0.968992  1.287962  0.052434    7.986867
150      (volatile acidity_high)               (quality_low)            0.235147            0.864290  0.225141    0.957447  1.107784  0.021905    3.189181
208           (citric acid_high)                    (pH_low)            0.248906            0.757974  0.237023    0.952261  1.256325  0.048359    5.069813
174                    (pH_high)           (citric acid_low)            0.242026            0.751094  0.230144    0.950904  1.266025  0.048359    5.069813
412  (total sulfur dioxide_high)               (quality_low)          

In [68]:
# hoher lift
"""Erkenntnis
- alkohol high -> quality high (geringe konfidenz aber hoher lift)
- fluechtige saeuren low -> quality_high (geringe konfidenz, aber hoher lift)
- viele sulfate -> quality high
==> nicht auschlaggebend, aber macht guten wein wahrscheinlicher, als 2./3. auswahlkriterium

2 Klassen:
- viel citronensaeure -> quality high (gerine konfidenz)
"""
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules.sort_values(by="lift", ascending=False, inplace=True)
print(rules.head(20))

                     antecedents                  consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction
48          (fixed acidity_high)           (citric acid_high)            0.247655            0.248906  0.167605    0.676768  2.718974  0.105962    2.323698
49            (citric acid_high)         (fixed acidity_high)            0.248906            0.247655  0.167605    0.673367  2.718974  0.105962    2.303334
483               (alcohol_high)               (quality_high)            0.237649            0.135710  0.085679    0.360526  2.656597  0.053427    1.351565
482               (quality_high)               (alcohol_high)            0.135710            0.237649  0.085679    0.631336  2.656597  0.053427    2.067878
67                (density_high)         (fixed acidity_high)            0.250156            0.247655  0.156973    0.627500  2.533769  0.095021    2.019719
66          (fixed acidity_high)               (density_high)   

In [73]:
# lift near 1
""" Erkenntis
Leider Attribute findbar die keinen Einfluss auf gut/schlechte Qualitaet haben.
=> So notieren und zeigen, dass man an lift near 1 gedacht hat

2 Klassen (Mit max_len2):
hoher restzucker, wenig fluechtige, .... -> keinen Einfluss auf qualitaet low
"""
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules = rules[rules.lift.between(0.95,1.05)]
rules = rules[rules.consequents.apply(lambda c: bool(c.intersection({"quality_low","quality_mid","quality_high"})))]
rules.sort_values(by="lift", ascending=True, inplace=True)
print(rules)

                    antecedents    consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction
280       (residual sugar_high)  (quality_low)            0.222014             0.86429  0.185116    0.833803  0.964726 -0.006769    0.816559
118      (volatile acidity_low)  (quality_low)            0.764853             0.86429  0.639149    0.835650  0.966863 -0.021905    0.825737
394  (total sulfur dioxide_low)  (quality_low)            0.751094             0.86429  0.629769    0.838468  0.970123 -0.019395    0.840142
426               (density_low)  (quality_low)            0.749844             0.86429  0.637273    0.849875  0.983321 -0.010809    0.903978
352   (free sulfur dioxide_low)  (quality_low)            0.751094             0.86429  0.640400    0.852623  0.986501 -0.008763    0.920833
449                    (pH_low)  (quality_low)            0.757974             0.86429  0.647280    0.853960  0.988048 -0.007830    0.929267
306          

In [76]:
# lift low
""" Erkenntis
- wenig freie sauere -> unwahrscheinlicher dass wein schlecht
- viele sulfate -> unwahrscheinlicher dass wein schlecht
- mittlere dicht -> hoher alkohol gehalt unwahrscheinlicher

2 Klassen (und max_len=2)
- wenig alk -> guter wein unwahrscheinlicher
- und viel mehr
"""
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules = rules[rules.consequents.apply(lambda c: bool(c.intersection({"quality_low","quality_mid","quality_high"})))]
rules.sort_values(by="lift", ascending=True, inplace=True)
print(rules.head(30))

                     antecedents     consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction
479                (alcohol_low)  (quality_high)            0.762351             0.13571  0.050031    0.065628  0.483587 -0.053427    0.924995
467              (sulphates_low)  (quality_high)            0.758599             0.13571  0.064415    0.084913  0.625699 -0.038534    0.944490
481               (alcohol_high)   (quality_low)            0.237649             0.86429  0.151970    0.639474  0.739883 -0.053427    0.376421
187            (citric acid_low)  (quality_high)            0.751094             0.13571  0.076298    0.101582  0.748524 -0.025633    0.962013
472             (sulphates_high)   (quality_low)            0.241401             0.86429  0.170106    0.704663  0.815309 -0.038534    0.459509
43           (fixed acidity_low)  (quality_high)            0.752345             0.13571  0.083802    0.111388  0.820782 -0.018298    0.972630